### SETUP

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%pip install ultralytics
%pip install deep_sort_realtime
%pip install gdown

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 84.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 63.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 90.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

In [ ]:
# Download and unzip pigs-tracking.zip
!gdown https://drive.google.com/uc?id=1PJyZedDyN06WiEGs0KTSWs7VcHuhNBGD -O /content/pigs-tracking.zip
!unzip /content/pigs-tracking.zip -d /content/videos

# Download and unzip pigs-yolov8-weights.zip
!gdown https://drive.google.com/uc?id=1YTbpI-m27oDJdGivKA1x01MA37Kv8_5- -O /content/pigs-yolov8-weights.zip
!unzip /content/pigs-yolov8-weights.zip -d /content

Downloading...
From (original): https://drive.google.com/uc?id=1PJyZedDyN06WiEGs0KTSWs7VcHuhNBGD
From (redirected): https://drive.google.com/uc?id=1PJyZedDyN06WiEGs0KTSWs7VcHuhNBGD&confirm=t&uuid=98be0e78-48ec-456f-9158-081cc9752f39
To: /content/pigs-tracking.zip
100% 1.18G/1.18G [00:59<00:00, 19.9MB/s]
Archive:  /content/pigs-tracking.zip
  inflating: /content/videos/Test Vid 1.mp4  
  inflating: /content/videos/Test Vid 2.mp4  
  inflating: /content/videos/Test Vid 3.mp4  
  inflating: /content/videos/Test Vid 5.mp4  
  inflating: /content/videos/Test Vid 8.mp4  
  inflating: /content/videos/Test Vid 6.mp4  
  inflating: /content/videos/Test Vid 7.mp4  
Downloading...
From (original): https://drive.google.com/uc?id=1YTbpI-m27oDJdGivKA1x01MA37Kv8_5-
From (redirected): https://drive.google.com/uc?id=1YTbpI-m27oDJdGivKA1x01MA37Kv8_5-&confirm=t&uuid=6b1dfb44-682b-4bd3-8e54-2c40054912c0
To: /content/pigs-yolov8-weights.zip
100% 169M/169M [00:10<00:00, 16.1MB/s]
Archive:  /content/pigs-yol

### Tracking Pig and Assigning ID

In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
import matplotlib.pyplot as plt
import matplotlib

matplotlib.use('Agg')

from deep_sort_realtime.deepsort_tracker import DeepSort

print("YOLOv8 model loaded successfully!")

model = YOLO('/content/v9/weights/best.pt')

tracker = DeepSort(
    max_iou_distance=0.7,
    max_age=30,
    n_init=3,
    nn_budget=100
)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
YOLOv8 model loaded successfully!


In [ ]:

video_path = "/content/videos/Test Vid 1.mp4"

cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print(f"Error: Could not open video file {video_path}")
    exit()

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

output_video_path = "/content/tracked_output.mp4"
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

frame_count = 0
while cap.isOpened():

    ret, frame = cap.read()
    if not ret:
        break

    results = model.predict(source=frame, verbose=False)

    detections = []

    pig_class_id = 0

    for r in results:
        boxes = r.boxes
        for box in boxes:

            x1, y1, x2, y2 = box.xyxy[0].tolist()
            conf = box.conf[0].item()
            cls = int(box.cls[0].item())

            if cls == pig_class_id and conf > 0.75:

                bbox_xywh = [x1, y1, x2 - x1, y2 - y1]
                detections.append((bbox_xywh, conf, cls))

    tracked_objects = tracker.update_tracks(detections, frame=frame)

    for track in tracked_objects:
        if not track.is_confirmed():
            continue

        track_id = track.track_id
        ltrb = track.to_ltrb()

        x1, y1, x2, y2 = int(ltrb[0]), int(ltrb[1]), int(ltrb[2]), int(ltrb[3])

        cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 4)

        text = f"ID: {track_id}"
        cv2.putText(frame, text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 4)

    out.write(frame)

    frame_count += 1

    if frame_count % 100 == 0:
        print(f"Processed {frame_count} frames.")

cap.release()
out.release()

print(f"\nTracking complete! Output video saved to: {output_video_path}")

Processed 100 frames.
Processed 200 frames.
Processed 300 frames.
Processed 400 frames.
Processed 500 frames.
Processed 600 frames.
Processed 700 frames.


### Converting to GIF for HTML

In [ ]:
from moviepy.editor import VideoFileClip
import os

video_path = "runs/detect/predict/Test Vid 1.avi"
video_path = "/content/tracked_output.mp4"
gif_path = "Test_Vid_1_output.gif"

if os.path.exists(video_path):
    with VideoFileClip(video_path) as clip:
        clip = clip.set_fps(10)
        clip = clip.resize((clip.w // 2, clip.h // 2))
        clip.write_gif(gif_path)

    print(f"Video successfully converted to GIF: {gif_path}")
else:
    print(f"Error: Video file not found at {video_path}")

In [ ]:
import base64
from IPython.display import HTML, display

gif_path = "Test_Vid_1_output.gif"

with open(gif_path, "rb") as gif_file:
    gif_data = gif_file.read()
gif_b64 = base64.b64encode(gif_data).decode('utf-8')

html_content = f"""
<img src="data:image/gif;base64,{gif_b64}" width="100%" height="auto">
"""

display(HTML(html_content))

In [ ]:
!cp /content/Test_Vid_1_output.gif /content/drive/MyDrive/Data/
!cp /content/tracked_output.mp4 /content/drive/MyDrive/Data/

### Refactored Code **∀** Videos

In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
import os
import base64
from IPython.display import HTML, display
from moviepy.editor import VideoFileClip
import matplotlib
matplotlib.use('Agg')
from deep_sort_realtime.deepsort_tracker import DeepSort
import gc
import pandas as pd # Import pandas

class VideoTracker:
    def __init__(self, model_path, pig_class_id=0, output_dir="tracked_videos_output"):
        print("Initializing VideoTracker...")
        self.model = YOLO(model_path)
        self.tracker = DeepSort(
            max_iou_distance=0.7,
            max_age=30,
            n_init=3,
            nn_budget=100
        )
        self.pig_class_id = pig_class_id
        self.output_dir = output_dir
        os.makedirs(self.output_dir, exist_ok=True)
        print(f"Output directory set to: {self.output_dir}")
        print("YOLOv8 model and DeepSORT tracker loaded successfully!")

    def process_video(self, video_path, display_gif=False):
        print(f"\n--- Processing video: {os.path.basename(video_path)} ---")
        cap = cv2.VideoCapture(video_path)

        if not cap.isOpened():
            print(f"Error: Could not open video file {video_path}")
            return

        frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = int(cap.get(cv2.CAP_PROP_FPS))

        video_filename = os.path.basename(video_path)
        output_video_name = f"tracked_{os.path.splitext(video_filename)[0]}.mp4"
        output_video_path = os.path.join(self.output_dir, output_video_name)

        # --- CSV Setup ---
        csv_filename = f"{os.path.splitext(video_filename)[0]}_tracking_data.csv"
        csv_path = os.path.join(self.output_dir, csv_filename)
        tracking_data = [] # List to store data for CSV

        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

        frame_count = 0
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            results = self.model.predict(source=frame, verbose=False)
            detections = []

            # Initialize a temporary list for current frame's detections before tracking
            current_frame_yolo_detections = []

            for r in results:
                boxes = r.boxes
                for box in boxes:
                    x1, y1, x2, y2 = box.xyxy[0].tolist()
                    conf = box.conf[0].item()
                    cls = int(box.cls[0].item())

                    if cls == self.pig_class_id and conf > 0.5:
                        bbox_xywh = [x1, y1, x2 - x1, y2 - y1]
                        detections.append((bbox_xywh, conf, cls)) # For DeepSORT input
                        # Store raw YOLO detection for potential later use if needed, not strictly for current CSV
                        current_frame_yolo_detections.append({'frame': frame_count, 'x1': x1, 'y1': y1, 'x2': x2, 'y2': y2, 'conf': conf, 'class_id': cls})


            tracked_objects = self.tracker.update_tracks(detections, frame=frame)

            dark_blue_color = (255, 0, 0)
            bbox_thickness = 4
            text_thickness = 3

            for track in tracked_objects:
                if not track.is_confirmed():
                    continue

                track_id = track.track_id
                ltrb = track.to_ltrb() # DeepSORT's output bounding box

                x1, y1, x2, y2 = int(ltrb[0]), int(ltrb[1]), int(ltrb[2]), int(ltrb[3])

                cv2.rectangle(frame, (x1, y1), (x2, y2), dark_blue_color, bbox_thickness)

                text = f"ID: {track_id}"
                cv2.putText(frame, text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, dark_blue_color, text_thickness)

                # --- Store tracked object details for CSV ---
                tracking_data.append({
                    'frame_id': frame_count,
                    'object_id': int(track_id),
                    'bbox_x1': x1,
                    'bbox_y1': y1,
                    'bbox_x2': x2,
                    'bbox_y2': y2,
                    'confidence': conf # This 'conf' is from the original detection that updated the track
                                      # DeepSORT doesn't output per-track confidence directly for each frame,
                                      # so we use the confidence from the associated detection.
                                      # This assumes the latest associated detection's confidence is what you want.
                })

            out.write(frame)
            frame_count += 1

            if frame_count % 200 == 0:
                print(f"Processed {frame_count} frames for {os.path.basename(video_path)}")
                gc.collect()

        cap.release()
        out.release()

        print(f"Tracking complete for {os.path.basename(video_path)}! Output video saved to: {output_video_path}")

        # --- Save tracking data to CSV ---
        if tracking_data:
            df = pd.DataFrame(tracking_data)
            df.to_csv(csv_path, index=False)
            print(f"Tracking data saved to: {csv_path}")
        else:
            print(f"No tracking data to save for {os.path.basename(video_path)}.")

        if display_gif:
            self._display_gif_from_video(output_video_path)

        # Explicitly delete references and collect garbage at end of video processing
        del cap, out, frame, results, detections, tracked_objects
        # No SAM in this version, so no 'frame_rgb'
        gc.collect()

    def _display_gif_from_video(self, video_path, gif_fps=10, resize_factor=0.5):
        gif_filename = f"{os.path.splitext(os.path.basename(video_path))[0]}.gif"
        gif_path = os.path.join(self.output_dir, gif_filename)

        if os.path.exists(video_path):
            try:
                with VideoFileClip(video_path) as clip:
                    clip = clip.set_fps(gif_fps)
                    clip = clip.resize(resize_factor)
                    clip.write_gif(gif_path, verbose=False)

                print(f"Video successfully converted to GIF: {gif_path}")

                with open(gif_path, "rb") as gif_file:
                    gif_data = gif_file.read()
                gif_b64 = base64.b64encode(gif_data).decode('utf-8')

                html_content = f"""
                <img src="data:image/gif;base64,{gif_b64}" width="100%" height="auto">
                """
                display(HTML(html_content))
            except Exception as e:
                print(f"Error converting video to GIF or displaying: {e}")
        else:
            print(f"Error: Video file not found at {video_path} for GIF conversion.")

# Example usage
tracker_instance = VideoTracker(
    model_path='/content/v9/weights/best.pt',
    pig_class_id=0,
    output_dir="/content/tracked_videos"
)

video_files = [
    "/content/videos/Test Vid 1.mp4",
    # "/content/videos/Test Vid 2.mp4",
    # "/content/videos/Test Vid 3.mp4",
    # "/content/videos/Test Vid 5.mp4",
    # "/content/videos/Test Vid 6.mp4",
    # "/content/videos/Test Vid 7.mp4",
    # "/content/videos/Test Vid 8.mp4"
]

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


  if event.key is 'enter':



Initializing VideoTracker...
Output directory set to: /content/tracked_videos
YOLOv8 model and DeepSORT tracker loaded successfully!


In [ ]:
for vid_path in video_files:
    tracker_instance.process_video(vid_path, display_gif=False)
    gc.collect()  # Manual garbage collection


--- Processing video: Test Vid 1.mp4 ---
Processed 200 frames for Test Vid 1.mp4
Processed 400 frames for Test Vid 1.mp4
Processed 600 frames for Test Vid 1.mp4
Processed 800 frames for Test Vid 1.mp4
Processed 1000 frames for Test Vid 1.mp4
Processed 1200 frames for Test Vid 1.mp4
Processed 1400 frames for Test Vid 1.mp4
Tracking complete for Test Vid 1.mp4! Output video saved to: /content/tracked_videos/tracked_Test Vid 1.mp4
Tracking data saved to: /content/tracked_videos/Test Vid 1_tracking_data.csv


### Export **∀** Videoz

In [ ]:
!cp -r /content/tracked_videos /content/drive/MyDrive/Data

cp: cannot create directory '/content/drive/MyDrive/Data': No such file or directory


In [ ]:
!rm -rf /content/tracked_videos